* __Author__: Fan Meng, SiYuan Chen, YingYing Ye, HaiLong Jiang
* __Date__: August 26th, 2023

- <a>Abstract</a>
- <a>1. Introduction</a>
    - <a>1.1 Background</a>
    - <a>1.2 Problem Statement</a>
    - <a>1.3 Data Description</a>
    - <a>1.4 Evaluation Metric</a>
- <a>2. Imports and Tools function</a>
- <a>3. Data</a>
    - <a> 3.1 Reading Training set data
    - <a> 3.2 Data augmentation
    - <a> 3.3 Dataset split
    - <a> 3.4 Normalization and segmentation of training set data
    - <a> 3.5 Dataset and Dataloader in torch
    - <a> 3.6 Normalization and segmentation of validation set data
- <a>4. Model training</a>
    - <a> 4.1 Prepare model
    - <a> 4.2 Verify model
    - <a> 4.3 Optimizer preparation
    - <a> 4.4 Metric Preparation
    - <a> 4.5 Training process and validation process
    - <a> 4.6 DTW

- <a>5. Model prediction</a>
    - <a> 5.1 Reading Test set data 
    - <a> 5.2 Normalization and segmentation of test set data
    - <a> 5.3 Dataset and dataloader in test set
    - <a> 5.4 Overload Model and JSON file
    - <a> 5.5 Model prediction
    - <a> 5.6 Save the results in a zip package format

<a></a>

## Abstract

Well log measurements are continuous records of indirectly measured formation properties and the main input for constructing static and dynamic reservoir models. In fact, not all well logging data of a single well are  measured at once, high-precision depth matching of well logging data is crucial for later rock physics interpretation and machine learning correlation extraction. We have developed a recurrent neural network structure based on multiple wavelet decomposition network and connected it with DTW technology to solve the depth alignment problem of well log. multiple wavelet decomposition network blocks are used for extracting frequency information from logging data, recurrent networks are used for capturing depth sequences information, DTW is used for depth matching and correction, the neural networks are trained using four different type of well logging measurements (GR, RHOB, NPHI, RD). Among them, GR as the depth reference. By comparing the submission results obtained from different network structure parameters, our best performance is NMSE=0.3148 and MAD=21.7658, which is a certain improvement compared to the baseline (NMSE=0.3959, MAD=25.9585), proving that the method has a some correction effect.

## <a>1. Introduction</a>

### <a>1.1 Background</a>

Well logs are crucial data source for the oil and gas industry, providing key information about subsurface formations and geological characteristics. However, the data collected through well logs can often be affected by borehole environments, such as the sticking and slipping of logging tools, resulting in inaccurate and unreliable results. Manual depth shift of well logs is time-consuming and subject to the expertise of interpreters. With the development of the petroleum industry, the importance of accurate logging data for making decisions, optimizing production, and reducing costs is becoming increasingly prominent.

In the traditional process, logging tools are lifted up in the borehole and collected rock physical response information of the formation based on a certain sampling interval. Electrical Wireline Logging(EWL) is widely used due to its high accuracy, Due to the complex environment in the formation and the limitations of logging tools, the original measured wireline logging data often has misalignment or offset, resulting in inaccurate depth. This depth migration may have a serious impact on oil well exploration and production processes, thereby reducing exploration and production efficiency.

### <a>1.2 Problem Statement</a>

The data for this research is sourced from the 2023 Machine Learning Competition "Automatic Well-Log Depth Shift with Data-Driven Methods" organized by the Petrophysics Data Driven Analysis(PDDA) Interest Group of Society of Petrophysicists and Well Log Analysts(SPWLA), https://github.com/pddasig/Machine-Learning-Competition-2023.

The objective of this competition is to develop a data-driven machine learning model that uses GR log as a reference to translate, stretch, and compress misaligned logging curves. The training and validation data were obtained from well logs of 9 wells in the same oilfield that were deeply corrected by petrologists, and then tested and scored from the other 3 wells in same oilfield. The submission should include the shifted well logs and the shifted depths. The evaluation of the competition is scored based on the normalized mean squared error (NMSE) of well-log prediction and the Mean absolute deviation (MAD) of depth shift prediction.

### <a>1.3 Data Description</a>

Training data for each well. Well logs are aligned by a senior petrophysicist. we need to augment the data in order to train the model which we will talk about at `Data augmentation and dataset preprocessing`. 

The test data has all features in the train dataset, but they are not aligned (raw), we will need to predict the corrected well logs and associated depth shift. The Predictor Varibles and Target Variables shown as follow:

Predictor Varibles in ./data/train/aligned_well_*.csv and ./data/test/aligned_well_*.csv:
- DEPT - Depth, unit in feet
- GR - **Reference Gamma Ray**, unit in API
- RHOB - **Aligned Density Log**, unit in Gamma per cubic centimeter
- NPHI - **Aligned Neutron Porosity**, unit in dec
- RD - **Aligned Deep Resistivity**, unit in Ohm.m

Target Variables:
- RHOB_pred - **Corrected Density Log**, unit in Gram per cubic centimeter
- NPHI_pred - **Corrected Neutron Porosity**, unit in dec
- RD_pred - **Corrected Deep Resistivity**, unit in Ohm.m
- RHOB_dept_pred - **Prediction of the Actual Depth of Raw Density Log**, unit in feet
- NPHI_dept_pred - **Prediction of the Actual Depth of Raw Neutron Porosity**, unit in feet
- RD_dept_pred - **Prediction of the Actual Depth of Raw Deep Resistivity**, unit in feet

The final submission result needs to be reflected in a compressed package named `dreamstar_submission_1.zip`. The submission file must be a zip file with three .csv files, which include your predictions of corrected/shfited well logs and their depth shift. 

### <a>1.4 Evaluation Metric</a>
Submissions are evaluated based on normalized mean squared error(NMSE) of corrected well logs and mean absolute deviation (MAD) of depth shift prediction, the final score will be rank transformed and averaged to avoid the scaling of different metrics. 

$$NMSE = \frac{1}{mn}\frac{1}{Var(\mathbf{y})}\sum_{i=1}^{m}\sum_{j=1}^{n}(\hat{\mathbf{y_{i,j}}} - \mathbf{y_{i,j}})^{2}$$

where
- $\hat{y_{i,j}}$ the prediction (RHOB_pred, NPHI_pred, RD_pred) of the **values** for shifted well log j, sample i. $y_{i,j}$ is the actual **values** of the well log j for sample i shifted by a petrophysicist. 
- $m$ is the sample size.
- $n$ is the number of well logs (RHOB, NPHI, RD): 3.
- $Var$ is the variance.

$$MAD = \frac{1}{mn}\sum_{i=1}^{m}\sum_{j=1}^{n}|\hat{\mathbf{d_{i,j}}} - \mathbf{d_{i,j}}| $$

where
- $\hat{d_{i,j}}$ is the prediction (RHOB_dept_pred, NPHI_dept_pred, RD_dept_pred) of **depth shift** for raw well log j, sample i. $d_{i,j}$ is the actual **depth shift of raw well logs** by a petrophysicist. 
- $m$ is the sample size.
- $n$ is the number of well logs (RHOB, NPHI, RD): 3.

## <a>2. Imports and Tools function</a>
> Statement: Our original code is Python scripts, but in order to facilitate readers' understanding of the workflow, we have chosen to use jupyter to reorganize our code, retaining only the valuable parts.

We use basic venv to establish our Python virtual environment, use Pytroch as the training framework for neural networks, and use argparse to save and change multiple parameters.

### 2.1 Imports

In [1]:
# install package for pytroch
# ! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
# ! pip install pandas

In [3]:
# ! pip install scipy

In [4]:
# ! pip install tqdm

In [5]:
# install package for DTW
# ! pip install tsaug
# ! pip install dtwalign

In [6]:
# import libraries
import json
import argparse # used to processing Command Line Parameters
import datetime
import os
import torch
import glob # use wildcards to search for files
import pandas as pd
import numpy as np
from numpy import array, ndarray
from scipy import interpolate

from tqdm import tqdm # used to load progress bar
import torch.nn as nn
from torch import as_tensor, tensor
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import shutil # used to zip predictions for submission

import sys
import time, numbers

### 2.2 Tools Function
>Tools Function involves some common tool functions throughout the project, including:
- writing log files
- creating directory
- timer
- converting command-line parameters to JSON files

In [7]:
# writing log files
# This function is used to save some important print logs
def pprint(log_file, *text):
    # print with UTC+8 time
    time = '[' + str(datetime.datetime.utcnow() +
                     datetime.timedelta(hours=8))[:19] + '] -'
    print(time, *text, flush=True)
    if log_file is None:
        return
    with open(log_file, 'a', encoding='utf-8_sig') as f:
        print(time, *text, flush=True, file=f)

In [8]:
# creating directory
# if the folder not exists, create it
def dir_exist(dirs):
    if not os.path.exists(dirs):
        os.makedirs(dirs)

In [9]:
# timer
class Timer:  # @save
    """Record multiple running times"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start Timer"""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in the list"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return average time"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """Total return time"""
        return sum(self.times)

    def cumsum(self):
        """Return cumulative time"""
        return np.array(self.times).cumsum().tolist()

In [10]:
# converting command-line parameters to JSON files
# Used for conversion between argparse (parameter namespace) and JSON, achieving the effect of parameter storage and overloading
# 用于argparse(参数命名空间)和json之间的转换，达到参数储存和重载的效果
def argparse_json(args=None, setting=None, mode=None, info=None):
    dir_path = os.path.join(args.checkpoints, setting) + '/json/'
    dir_exist(dir_path)

    if mode == 'save':
        json_file = dir_path + '{}-{}.json'.format(args.model, info)
        # save args to json - 将参数储存在JSON文件中
        with open(json_file, 'w') as f:
            json.dump(vars(args), f)

    if mode == 'resume':
        try:
            json_file = dir_path + '{}.json'.format(info[:-4])
            # Read parameters from JSON file and recreate parser object - 从JSON文件中读取参数并重新创建解析器对象
            with open(json_file, 'r') as f:
                args_dict = json.load(f)
            # Convert the dictionary object to an argparse. Namespace object and parse the command line parameters - 将字典对象转换为argparse.Namespace对象，并解析命令行参数
            new_args = argparse.Namespace(**args_dict)
            new_args.resume = info
            return new_args
        except Exception as e:
            print('-' * 80)
            print("can't find json file, use default in main - 未发现模型对应的json文件, 使用main.py中的默认值")
            print(f'{e}')
            print('-' * 80)
            return args


###  2.3 Namespace
> The purpose of this function is to save all variable parameters to ensure their applicability to various models and parameter combinations

Note: If you want to train model from the beginning, you need to set the default value of the resume parameter to ' ' in get_args_parser function


In [11]:
# The purpose of this function is to save all variable parameters to ensure their applicability to various models and parameter combinations
def get_args_parser():
    parser = argparse.ArgumentParser(description='[Automatic Well-Log Depth Shift]')
    parser.add_argument('--data_path', type=str, default='./data', help='root path of the data file')
    parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')
    # data parameters
    parser.add_argument('--seq_len', type=int, default=224, help='input sequence length of features')  # 224
    parser.add_argument('--label_len', type=int, default=224, help='prediction sequence length of target')  # 224
    parser.add_argument('--shift_len', type=int, default=0, help='shift length between input feature and output label')
    parser.add_argument('--train_columns', type=list, default=['DEPT', 'GR', 'RHOB', 'NPHI', 'log_RD'], help='the columns name of input feature')
    parser.add_argument('--label_columns', type=list, default=['RHOB_pred', 'NPHI_pred', 'log_RD_pred', 'RHOB_dept_pred', 'NPHI_dept_pred', 'log_RD_dept_pred'],
                        help='the columns name of output label')
    parser.add_argument('--data_augment_num', type=int, default=3, help='The number of data augmentations')
    parser.add_argument('--scale', type=bool, default=True, help='scale input data or not')
    parser.add_argument('--scaler', type=str, default='Normalization', help='options:[Normalization, Standardization]')
    parser.add_argument('--train_data_mean', type=dict, help='Mean value of training set data')
    parser.add_argument('--train_data_std', type=dict, help='Variance of training set data')
    parser.add_argument('--train_data_max', type=dict, help='Max of training set data')
    parser.add_argument('--train_data_min', type=dict, help='Min of training set data')
    # model parameter
    # parser.add_argument('--use_deep_learning', type=bool, default=True, help="use deep learning or not")  # 11
    # parser.add_argument('--dtw_after_predict', type=bool, default=True, help="use DTW after regression prediction") # 11
    parser.add_argument('--model', type=str, default='mWDN_gru', 
                        help="These are models that have been tried before: [seq_rnn, seq_brnn, seq_lstm, seq_blstm, seq_gru, seq_bgru, "
                             "u_net, wavelet_lstm, mWDN_lstm, mWDN_gru, mWDN_blstm, mWDN_bgru"
                             "Linear, SVM, RF, autoML]")
    parser.add_argument('--model_para_num', type=int, help='the number of the model parameters')
    # mWDN_gru and mWDN_lstm parameter
    parser.add_argument('--mWDN_level', type=int, default=3, help='the level of Multilevel Wavelet Decomposition Network')
    parser.add_argument('--mWDN_rnn_hidden_size', type=int, default=256, help='number of rnn hidden units per layer')
    parser.add_argument('--mWDN_rnn_num_layers', type=int, default=1, help='number of rnn layers')
    parser.add_argument('--mWDN_fcn_hidden_size', type=int, default=256, help='number of fcn hidden units')
    parser.add_argument('--mWDN_alpha', type=float, default=0.5, help='hyper-parameters to adjust the degree of '
                             'control over the mWDN matrix about low frequency')
    parser.add_argument('--mWDN_beta', type=float, default=0.5, help='hyper-parameters to adjust the degree of '
                             'control over the mWDN matrix about high frequency')
    # DTW parameter
    parser.add_argument('--dtw_method', type=str, default='dtwalign', help='options: [dtwalign, dtwalign_ddtw]')
    parser.add_argument('--dtw_window_size', type=int, default=180, help='options: [100/140/180/220/260]')
    parser.add_argument('--dtw_window_type', type=str, default="sakoechiba", help='options: [sakoechiba, itakura]')
    parser.add_argument('--dtw_step_pattern', type=str, default="symmetricP05",
                        help='options: [symmetricP05, asymmetricP05, symmetricP2, asymmetricP2, symmetric1, typeIb, typeIIa]')
    # deeplearning - train parameter
    parser.add_argument('--train_epochs', type=int, default=100, help='the number of train epochs')
    parser.add_argument('--batch_size', type=int, default=128, help='batch size of train input data')
    parser.add_argument('--optimizer', type=str, default='adam', help='options:[sgd, adam]')
    parser.add_argument('--lr', type=float, default=0.001, help='optimizer learning rate')
    parser.add_argument('--patience', type=int, default=5, help='early stopping')
    # predict parameter
    parser.add_argument('--resume', type=str, default='mWDN_gru-0.00596.pth', help='path of model to resume')  # resume-tag  # set '' to train new model
    parser.add_argument('--output_path', type=str, default='./results/', help='root path of the result')
    # other parameters
    parser.add_argument('--log_file', type=str, default='run.log',
                        help='Training log file, The file name will be modified according to the model situation')
    parser.add_argument('--seed', type=int, default=13, help='random seed')
    parser.add_argument('--use_gpu', type=bool, default=True if torch.cuda.is_available() else False, help='use gpu or not')
 
    return parser

In [12]:
# Prepare the namespace and fix the random seed
parser = get_args_parser()
args = parser.parse_args(args=['--seed', '12'])
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [13]:
# define log and model path
setting = '{}-seq({})-label({})'.format(args.model, args.seq_len, args.label_len)
log_file_path = os.path.join(args.output_path, 'train_log')  # Log file save directory
dir_exist(log_file_path)  # Create if directory does not exist
args.log_file = os.path.join(log_file_path, '{0}.log'.format(setting))  # Overwrite Log File Save Path

In [14]:
# global variable - 全局变量
num = args.data_augment_num # Augmentation frequency - 增广次数
seed = args.seed
label_columns = args.label_columns
seq_len = args.seq_len
pred_len = args.label_len
shift_len = args.shift_len

## 3.Data
> This chapter is conducting data preparation work, including obtaining datasets, expanding datasets, splitting datasets(for obtaining training and validation sets), normalizing and sequence segmenting the data to prepare for the format required by the recurrent neural network in the future.

### 3.1 Reading training set data to obtain dataframe - 读取训练集数据获得dataframe

In [15]:
# Merge the data from nine wells into one DataFrame to prepare for the following segmentation training and validation sets
# 将九口井的数据合并在一个DataFrame里，为下面切分训练集和验证集做准备
current_path = os.getcwd()  # absolute path of the current file - 获取当前文件的绝对路径
data_path = os.path.join(current_path, 'data/train')  # Obtain the path of the data file - 获取数据文件的路径
files = glob.glob(f'{data_path}//aligned_well_0*.csv')
df = []
y_feat = ['RHOB', 'NPHI', 'log_RD']
for i in files:
    df0 = pd.read_csv(i)
    df0['log_RD'] = np.log10(df0['RD'])
    df0.drop('RD', axis=1, inplace=True)
    for feat in y_feat:
        df0[f'{feat}_pred'] = df0[f'{feat}'].copy()
    for feat in y_feat:  # Two cycles to ensure order - 两次循环是为了保证顺序
        df0[f'{feat}_dept_pred'] = df0['DEPT'].copy()
    df0['wellnum'] = i.split('_')[-1].split('.')[0]
    df.append(df0.copy())
df_c = pd.concat(df)

In [16]:
# Output the DataFrame after merging nine wells - 输出九口井合并之后的DataFrame
df_c

,DEPT,GR,RHOB,NPHI,log_RD,RHOB_pred,NPHI_pred,log_RD_pred,RHOB_dept_pred,NPHI_dept_pred,log_RD_dept_pred,wellnum
0,411.0,94.0070,2.2421,0.4708,2.492080,2.2421,0.4708,2.492080,411.0,411.0,411.0,01
1,411.5,95.0090,2.2631,0.4757,2.861019,2.2631,0.4757,2.861019,411.5,411.5,411.5,01
2,412.0,96.1010,2.2757,0.4510,2.989289,2.2757,0.4510,2.989289,412.0,412.0,412.0,01
3,412.5,95.6830,2.2726,0.4282,2.989289,2.2726,0.4282,2.989289,412.5,412.5,412.5,01
4,413.0,93.0250,2.2764,0.4085,2.989289,2.2764,0.4085,2.989289,413.0,413.0,413.0,01
...,...,...,...,...,...,...,...,...,...,...,...,...
10219,5606.5,34.5635,2.4937,0.1164,2.382242,2.4937,0.1164,2.382242,5606.5,5606.5,5606.5,09
10220,5607.0,38.6881,2.5026,0.1132,2.142852,2.5026,0.1132,2.142852,5607.0,5607.0,5607.0,09
10221,5607.5,41.0457,2.5190,0.1132,1.942452,2.5190,0.1132,1.942452,5607.5,5607.5,5607.5,09
10222,5608.0,43.9938,2.5309,0.1148,1.922806,2.5309,0.1148,1.922806,5608.0,5608.0,5608.0,09


### 3.2 Data augmentation - 数据增广
>Data augmentation involves offsetting, compressing, and stretching the original data to create the training and validation sets required for training. Randomly sampling 60% of the data as anchor points, using the corresponding relationships in the anchor index comparison table, the original data is stretched, compressed, and translated to varying degrees.

![Data_augmentation](imgs/fig.1.Data_augmentation.png)

In [17]:
# Randomly extend and compress the target logging curve while maintaining the same starting and ending depths
# 随机伸展和压缩目标测井曲线，并保持起始深度和结束深度不变
def random_stretch(curve, depth0, ank_p=0.6, seed=10):
    """
    Randomly stretch and compress the target logging curve while maintaining the same starting and ending depths

    Parameters

    ----------
    curve : ndarray
        Value of target logging curve.
    depth0 : ndarray
        Depth of target logging curve.
    ank_p : float
        The proportion of randomly sampled anchor points, this value may not be quite certain yet.

    Returns
    -------
    new_curve : ndarray
        New logging curve values.
    new_depth : ndarray
        New logging curve depth.

    """
    length = curve.shape[0]
    assert depth0.shape[0] == curve.shape[0], "depth and curve do not match!"
    depth = depth0.copy()
    # find location after perturbation
    n = int(length * ank_p)
    np.random.seed(seed)  # Set random seeds - 设定随机种子
    anchor_points0 = np.sort(np.random.choice(np.arange(0, length), n, replace=False))
    anchor_points1 = np.sort(np.random.choice(np.arange(0, length), n, replace=False))

    depth1 = np.interp(depth, depth[anchor_points0], depth[anchor_points1], left=-99999, right=99999)
    good = (-99999 < depth1) * (depth1 < 99999)
    depth1[depth1 == 99999] = depth1[good][-1] - depth[good][-1] + depth[depth1 == 99999]
    depth1[depth1 == -99999] = depth1[good][0] - depth[good][0] + depth[depth1 == -99999]
    f = interpolate.interp1d(depth1, curve, fill_value=(curve[0], curve[-1]), bounds_error=False, kind=1)
    new_curve = f(depth0) + np.random.normal(0, 0.0001, curve.shape)  # values of perturbed log - 数值扰动
    new_depth = np.interp(depth0, depth1, depth0, left=depth0[0], right=depth0[-1])  # truth depth of perturbed log
    return new_curve, new_depth

In [18]:
# Through random_stretch function amplifies the data for each well's (RHOB, NPHI, Log_RD) three features in training set
# 通过random_stretch函数对每口井的(RHOB, NPHI, Log_RD)这三个特征进行数据增广
def curve_data_augmentation(df_well, y_feat, k, seed=10):
    df_well_temp = df_well.copy()
    random_seed = seed
    for i in df_well_temp['wellnum'].unique():
        idx = df_well_temp['wellnum'] == i
        df_well_temp.loc[idx, 'wellnum'] = i + f'_{k+1}'
        for j in y_feat:
            ank_p = round(np.random.uniform(0.7, 1), 1)
            random_seed = random_seed + 1
            df_well_temp.loc[idx, j], df_well_temp.loc[idx, j+'_dept_pred'] = \
                random_stretch(df_well_temp.loc[idx, j].values, df_well_temp.loc[idx, 'DEPT'].values, ank_p=ank_p, seed=random_seed)
    return df_well_temp

In [19]:
# Perform num rounds of augmentation on each well, ultimately obtaining data for num * 9 wells and saving it
# 对每口井分别进行num次增广，最终得到num * 9口井的数据并保存
data_path = os.path.join(current_path, 'stretch')  # Obtain the path of the data file - 获取数据文件的路径
dir_exist(data_path)
df = []
for i in range(num):
    seed = seed + 1
    df_temp = curve_data_augmentation(df_c, y_feat, i, seed=seed)
    df.append(df_temp.copy())
    # Output the results of single well random compression stretching transformation - corresponding to the output results
    # 输出单井随机压缩拉伸变换的结果 - 与输出结果对应
    df_temp = df_temp.copy()
    df_temp['log_RD'] = 10 ** (df_temp['log_RD'])
    df_temp['log_RD_pred'] = 10 ** (df_temp['log_RD_pred'])
    df_temp.rename(columns={'log_RD': 'RD', 'log_RD_pred': 'RD_pred', 'log_RD_dept_pred': 'RD_dept_pred'},
                    inplace=True)
    for j in df_temp['wellnum'].unique():
        df_temp[df_temp['wellnum'] == j].to_csv(f'{data_path}\\aligned_well_{j}_strech_{i+1}.csv', index=False)
df_c = pd.concat(df)
df_c.reset_index(drop=True, inplace=True)

In [20]:
# df_train_data represents the expanded dataset
# df_train_data表示增广之后的数据集
df_train_data = df_c  

### 3.3 Dataset split - 数据集切分

In [21]:
# Take the first 70% of the wells in the augmented dataset as the training set, and the last 30% as the validation set
# 取增广数据集的前70%的井作为训练集，后30%作为验证集
wells = df_train_data['wellnum'].unique()  # Using wellnum as segmentation method - 以井号为切分方式
wells.sort()  # Sort the wellnum, with maximum not exceeding 9 - 对井号进行排序，最大井号不超过9
n = num * 7
train_df = df_train_data[df_train_data['wellnum'].isin(wells[:n])].copy()
val_df = df_train_data[df_train_data['wellnum'].isin(wells[n:])].copy()

In [22]:
train_df.head(2)

,DEPT,GR,RHOB,NPHI,log_RD,RHOB_pred,NPHI_pred,log_RD_pred,RHOB_dept_pred,NPHI_dept_pred,log_RD_dept_pred,wellnum
0,411.0,94.007,2.241879,0.470824,2.492103,2.2421,0.4708,2.492080,411.0,411.00,411.0,01_1
1,411.5,95.009,2.263199,0.473442,2.861084,2.2631,0.4757,2.861019,411.5,411.25,411.5,01_1


In [23]:
val_df.head(2)

,DEPT,GR,RHOB,NPHI,log_RD,RHOB_pred,NPHI_pred,log_RD_pred,RHOB_dept_pred,NPHI_dept_pred,log_RD_dept_pred,wellnum
51796,550.0,92.741,2.492783,0.361357,0.871021,2.4928,0.3613,0.871047,550.0,550.0,550.0,08_1
51797,550.5,94.717,2.487911,0.361457,0.852728,2.4878,0.3646,0.852700,550.5,550.0,550.5,08_1


### 3.4 Normalization and segmentation of training set data - 训练集数据归一化、数据切分

In [24]:
# Normalization function and standardized function design - 归一化函数及标准化函数设计  https://www.zhihu.com/question/20467170
class Scaler:
    """    
    Including standardization and normalization, using mode for judgment
    包含了数据标准化及数据归一化处理，用mode进行判断
    """
    def __init__(self):
        pass

    @staticmethod
    def fit(df_data):
        if type(df_data) != pd.DataFrame:
            raise "df_data is not dataframe"
        mean = df_data.mean()
        std = df_data.std()
        max = df_data.max()
        min = df_data.min()
        return mean, std, max, min

    @staticmethod
    def transform(df_data, mean, std, max=None, min=None, mode='Standardization'):
        if mode == 'Standardization':
            standard_data = (df_data - mean) / std
        elif mode == 'Normalization':
            standard_data = (df_data - min) / (max - min)
        else:
            raise "please make sure the scaler mode"
        return standard_data

    @staticmethod
    def inverse_transform(data, mean=None, std=None, max=None, min=None, mode='Standardization'):
        if isinstance(data, pd.DataFrame):
            mean = mean[data.columns]
            std = std[data.columns]
            max = max[data.columns]
            min = min[data.columns]
        elif isinstance(data, torch.Tensor):
            try:
                data = data.cpu()
                data = data.numpy()
            except:
                data = data.detach()
                data = data.numpy()
            mean = mean.to_numpy()
            std = std.to_numpy()
            max = max.to_numpy()
            min = min.to_numpy()
        if data.shape[2] == 3:
            mean = mean[:3]
            std = std[:3]
            max = max[:3]
            min = min[:3]
        if mode == 'Standardization':
            inverse_data = (data * std) + mean
        elif mode == 'Normalization':
            inverse_data = data * (max - min) + min
        else:
            raise "please make sure the scaler mode"
        return inverse_data

In [25]:
# init scale and scaler - 初始化标准化的控制变量及函数
scale = args.scale
scaler = Scaler()

In [26]:
# obtain the DataFrame of train dataset
df_raw = train_df

In [27]:
# data normalization - 数据归一化
cols = list(df_raw.columns)
for col in label_columns:  # Remove target column - 去掉预测目标列
    cols.remove(col)
cols.remove('wellnum')  # Remove wellnum column - 去掉井名
df_raw = df_raw[['wellnum'] + cols + [x for x in label_columns]]  # Reorder columns-对列重新排序
cols_list = df_raw.columns[1:]  # Other columns names without wellnum - 不取井名 - 为了取归一化/标准化参数
df_data = df_raw[cols_list]

In [28]:
# Find the mean, variance, maximum, and minimum values of the first five columns of data
# 求前五列数据的均值、方差、最大值、最小值
train_data = df_data.iloc[:,:]
train_df_mean, train_df_std, train_df_max, train_df_min = scaler.fit(train_data)
# save the mean, variance, maximum, and minimum values calculated from the training set to the args (global parameter space)
# 将训练集计算得到的均值、方差、最大值和最小值储存于args全局参数空间中
args.train_data_mean, args.train_data_std = train_df_mean.to_dict(), train_df_std.to_dict()
args.train_data_max, args.train_data_min = train_df_max.to_dict(), train_df_min.to_dict()

In [29]:
# output the mean, variance, maximum and minimum values of the training set DataFrame
print("train_df_mean:",train_df_mean)
print("train_df_std:",train_df_std)
print("train_df_max:",train_df_max)
print("train_df_min:",train_df_min)

train_df_mean: DEPT                2823.193432
GR                   105.928636
RHOB                   2.485743
NPHI                   0.267164
log_RD                 0.911849
RHOB_pred              2.485289
NPHI_pred              0.267315
log_RD_pred            0.910886
RHOB_dept_pred      2827.711509
NPHI_dept_pred      2825.248832
log_RD_dept_pred    2826.225368
dtype: float64
train_df_std: DEPT                1304.769960
GR                    30.254012
RHOB                   0.131851
NPHI                   0.077829
log_RD                 0.391379
RHOB_pred              0.132835
NPHI_pred              0.077794
log_RD_pred            0.391397
RHOB_dept_pred      1305.584699
NPHI_dept_pred      1305.427222
log_RD_dept_pred    1305.205395
dtype: float64
train_df_max: DEPT                5743.000000
GR                   400.000000
RHOB                   3.045315
NPHI                   0.665037
log_RD                 4.697986
RHOB_pred              3.045400
NPHI_pred              0.665000

In [30]:
# Obtain data from different wells
# 获取不同井的数据
df_data_well_list = []
for wellnum in df_raw['wellnum'].unique():
    df_temp = df_raw[df_raw['wellnum'] == wellnum]
    df_data_well_list.append(df_temp)   

In [31]:
# Sequence segmentation of training set features and labels
# 对训练集特征和标签进行序列切分
def make_train_feat_seq_list(seq_list, all_len, feat, np_list, start_index, end_index, seq_len):
    for i in range(all_len):
        np_temp = feat.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[0:seq_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

def make_train_label_seq_list(seq_list, all_len, label, np_list, start_index, end_index, shift_len, pred_len):
    for i in range(all_len):
        np_temp = label.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[shift_len:shift_len + pred_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

In [32]:
# Standardize the training data, and then obtain the features, labels, and indices of the training data
# 首先对训练数据进行标准化，之后获得训练数据的特征、标签、和index
train_feat_seq_list = []
train_label_seq_list = []
train_index_seq_list = []
for df_data in df_data_well_list:
    if scale:
        df_data = scaler.transform(df_data[cols + [x for x in label_columns]],
                                        train_df_mean, train_df_std,
                                        train_df_max, train_df_min, mode=args.scaler)
    df_data['index'] = df_data.index
    start_index = df_data.index[0]
    end_index = df_data.index[-1]
    feat, label, index = df_data[cols], df_data[label_columns], df_data[['index']]
    # make sequence data - 构造序列数据
    feat_list = []
    label_list = []
    index_list = []
    if seq_len < shift_len + pred_len:
        all_len = shift_len + pred_len
    else:
        all_len = seq_len
    train_feat_seq_list = make_train_feat_seq_list(train_feat_seq_list, all_len, feat, feat_list,
                                            start_index, end_index, seq_len)
    train_label_seq_list = make_train_label_seq_list(train_label_seq_list, all_len, label, label_list,
                                                start_index, end_index, shift_len, pred_len)
    train_index_seq_list = make_train_feat_seq_list(train_index_seq_list, all_len, index, index_list,
                                                start_index, end_index, seq_len)

# Data connection between different wells - 不同井之间的数据相连
train_feat_seq_all = np.concatenate(train_feat_seq_list, axis=0)
train_label_seq_all = np.concatenate(train_label_seq_list, axis=0)
train_index_seq_all = np.concatenate(train_index_seq_list, axis=0)

In [33]:
# The segmented sequence data form, where 224 represents the sequence length, 
# 5 represents the dimension of input data features, and 6 represents the feature dimension of the predicted target
# 切分后的序列数据形式，224代表序列长度，5代表输入数据特征的维度，6代表预测目标的特征维度
train_feat_seq_all[0].shape, train_label_seq_all[0].shape, train_index_seq_all[0].shape

((224, 5), (224, 6), (224, 1))

### 3.5 Dataset and Dataloader in torch - torch中的Dataset 和Dataloader

In [34]:
# Iterative encapsulation of data - 对数据进行迭代封装
from torch.utils.data import Dataset
class WellDataset(Dataset):
    def __init__(self, df_feature, df_label, df_index, transform=None):
        # Determine whether the length of features, labels, and regression labels is consistent
        # 判断特征、标签、回归标签长度一致
        assert len(df_feature) == len(df_label)

        self.df_feature = df_feature
        self.df_label = df_label
        self.df_index = df_index

        self.T = transform
        self.df_feature = torch.tensor(self.df_feature, dtype=torch.float32)
        self.df_label = torch.tensor(self.df_label, dtype=torch.float32)

    def __getitem__(self, index):
        sample, target, df_idx = self.df_feature[index], self.df_label[index], self.df_index[index]
        if self.T:
            return self.T(sample), target, df_idx
        else:
            return sample, target, df_idx

    def __len__(self):
        return len(self.df_feature)

In [35]:
# Prepare train data adapted to neural networks through DataLoader
# 通过DataLoader准备与神经网络适配的train数据
train_dataset = WellDataset(train_feat_seq_all, train_label_seq_all, train_index_seq_all)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)

In [36]:
# Test the train_loader
# 测试一下train_loader
next(iter(train_loader))

[tensor([[[0.5111, 0.2419, 0.7142, 0.3190, 0.2698],
          [0.5112, 0.2325, 0.7106, 0.3187, 0.2595],
          [0.5113, 0.2392, 0.7011, 0.3174, 0.2578],
          ...,
          [0.5318, 0.2922, 0.7052, 0.3396, 0.2566],
          [0.5319, 0.2864, 0.7058, 0.3216, 0.2559],
          [0.5320, 0.2863, 0.7030, 0.3083, 0.2570]],
 
         [[0.3904, 0.2726, 0.7463, 0.4324, 0.2895],
          [0.3905, 0.2867, 0.7395, 0.4264, 0.2837],
          [0.3906, 0.3010, 0.7292, 0.4207, 0.2826],
          ...,
          [0.4111, 0.1890, 0.6589, 0.2746, 0.3791],
          [0.4112, 0.1967, 0.6421, 0.2691, 0.3885],
          [0.4113, 0.1858, 0.6342, 0.2598, 0.3874]],
 
         [[0.3251, 0.2466, 0.7078, 0.4383, 0.2674],
          [0.3252, 0.2379, 0.7109, 0.4368, 0.2678],
          [0.3253, 0.2309, 0.7252, 0.4408, 0.2681],
          ...,
          [0.3457, 0.2622, 0.7240, 0.4476, 0.2738],
          [0.3458, 0.2649, 0.7265, 0.4417, 0.2736],
          [0.3459, 0.2702, 0.7272, 0.4292, 0.2732]],
 
         .

### 3.6 Normalization and segmentation of validation set data - 验证集数据归一化、数据切分

In [37]:
# obtain the DataFrame of validation set
df_raw = val_df

In [38]:
# Normalization of validation set data - 验证集数据归一化
cols = list(df_raw.columns)
for col in label_columns:  # Remove target column - 去掉预测目标列
    cols.remove(col)
cols.remove('wellnum')  # Remove wellnum column - 去掉井名
df_raw = df_raw[['wellnum'] + cols + [x for x in label_columns]]  # Reorder columns-对列重新排序
cols_list = df_raw.columns[1:]  # Other columns names without wellnum - 不取井名 - 为了取归一化/标准化参数
df_data = df_raw[cols_list]

In [39]:
# Obtain data from different wells - 获取不同井的数据
df_data_well_list = []
for wellnum in df_raw['wellnum'].unique():
    df_temp = df_raw[df_raw['wellnum'] == wellnum]
    df_data_well_list.append(df_temp)

In [40]:
# Sequence segmentation of validation set features and labels - 对验证集特征和标签进行序列切分
def make_val_feat_seq_list(seq_list, all_len, feat, np_list, start_index, end_index, seq_len):
    for i in range(all_len):
        np_temp = feat.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[0:seq_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

def make_val_label_seq_list(seq_list, all_len, label, np_list, start_index, end_index, shift_len, pred_len):
    for i in range(all_len):
        np_temp = label.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[shift_len:shift_len + pred_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

In [41]:
# Sequence segmentation of validation set features and labels - 对验证数据进行标准化，并获得训练数据的特征、标签、和index
val_feat_seq_list = []
val_label_seq_list = []
val_index_seq_list = []
for df_data in df_data_well_list:
    if scale:
        df_data = scaler.transform(df_data[cols + [x for x in label_columns]],
                                        train_df_mean, train_df_std,
                                        train_df_max, train_df_min, mode=args.scaler)
    df_data['index'] = df_data.index
    start_index = df_data.index[0]
    end_index = df_data.index[-1]
    feat, label, index = df_data[cols], df_data[label_columns], df_data[['index']]
    # make sequence data - 构造序列数据
    feat_list = []
    label_list = []
    index_list = []
    if seq_len < shift_len + pred_len:
        all_len = shift_len + pred_len
    else:
        all_len = seq_len
    val_feat_seq_list = make_val_feat_seq_list(val_feat_seq_list, all_len, feat, feat_list,
                                            start_index, end_index, seq_len)
    val_label_seq_list = make_val_label_seq_list(val_label_seq_list, all_len, label, label_list,
                                                start_index, end_index, shift_len, pred_len)
    val_index_seq_list = make_val_feat_seq_list(val_index_seq_list, all_len, index, index_list,
                                                start_index, end_index, seq_len)

# Data connection between different wells - 不同井之间的数据相连
val_feat_seq_all = np.concatenate(val_feat_seq_list, axis=0)
val_label_seq_all = np.concatenate(val_label_seq_list, axis=0)
val_index_seq_all = np.concatenate(val_index_seq_list, axis=0)

In [42]:
# Prepare val data adapted to neural networks through DataLoader - 通过DataLoader准备与神经网络适配的val数据
val_dataset = WellDataset(val_feat_seq_all, val_label_seq_all, val_index_seq_all)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)

In [43]:
# Test val_ Loader - 测试一下val_loader
next(iter(val_loader))

[tensor([[[0.0290, 0.2175, 0.6894, 0.5355, 0.2879],
          [0.0291, 0.2225, 0.6866, 0.5356, 0.2845],
          [0.0292, 0.2253, 0.6841, 0.5404, 0.2835],
          ...,
          [0.0496, 0.2231, 0.7047, 0.4127, 0.3170],
          [0.0497, 0.2214, 0.7013, 0.4161, 0.3162],
          [0.0498, 0.2217, 0.6955, 0.4177, 0.3155]],
 
         [[0.0291, 0.2225, 0.6866, 0.5356, 0.2845],
          [0.0292, 0.2253, 0.6841, 0.5404, 0.2835],
          [0.0293, 0.2209, 0.6813, 0.5288, 0.2835],
          ...,
          [0.0497, 0.2214, 0.7013, 0.4161, 0.3162],
          [0.0498, 0.2217, 0.6955, 0.4177, 0.3155],
          [0.0499, 0.2234, 0.6888, 0.4213, 0.3147]],
 
         [[0.0292, 0.2253, 0.6841, 0.5404, 0.2835],
          [0.0293, 0.2209, 0.6813, 0.5288, 0.2835],
          [0.0294, 0.2156, 0.6774, 0.5156, 0.2811],
          ...,
          [0.0498, 0.2217, 0.6955, 0.4177, 0.3155],
          [0.0499, 0.2234, 0.6888, 0.4213, 0.3147],
          [0.0500, 0.2248, 0.6825, 0.4381, 0.3140]],
 
         .

##  4. model training - 模型训练
>Predict the data through a multi-level wavelet decomposition network, and then align the sequences using the DTW algorithm.
![Alt text](%E5%9B%BE%E7%89%871-1.png)

### 4.1 Prepare model
> Multilevel Discrete Wavelet Decomposition (MDWD) is a wavelet based discrete signal analysis method, which can extract multilevel time-frequency features from a time series by decomposing the series as low and high frequency sub-series level by level(Mallat, 1989).Perform multi-level wavelet decomposition on the original signal, then place the high-frequency signal of each level and the low-frequency signal of the last level into the GRU network for calculation, and then input the obtained results into a fully connected network with two layers.

![model_structure](imgs/fig.2.model_structure.png)

In [44]:
# Initialize model parameters
# 初始化模型参数
class ModelMaker():
    def __init__(self, args):
        self.args = args
        self.input_size = len(args.train_columns)
        self.output_size = len(args.label_columns[:3])
        
model_maker = ModelMaker(args)

In [45]:
import pandas as pd
import torch
from torch import as_tensor, Tensor
import torch.nn as nn
import torch.nn.functional as F
import hashlib, itertools, types, inspect, functools, random, time, math, bz2, typing, numbers, string
from numpy import array, ndarray

In [46]:
# Fundamentals of Multilevel Wavelet Network Decomposition
# 多级小波网络分解基础
class WaveBlock(nn.Module):  
    def __init__(self, seq_len, wavelet=None, alpha=0.5, beta=0.5):
        super(WaveBlock, self).__init__()
        device = torch.device(torch.cuda.current_device()) if torch.cuda.is_available() else torch.device("cpu")
        self.alpha = alpha
        self.beta = beta
        if wavelet is None:
            self.h_filter = [-0.2304, 0.7148, -0.6309, -0.028, 0.187, 0.0308, -0.0329, -0.0106]
            self.l_filter = [-0.0106, 0.0329, 0.0308, -0.187, -0.028, 0.6309, 0.7148, 0.2304]
        else:
            try:
                import pywt
                # https: // blog.csdn.net / wsp_1138886114 / article / details / 116780542
            except ImportError:
                raise ImportError("You need to either install pywt to run mWDN or set wavelet=None")
            w = pywt.Wavelet(wavelet)
            self.h_filter = w.dec_hi
            self.l_filter = w.dec_lo

        self.mWDN_H = nn.Linear(seq_len, seq_len)
        self.mWDN_L = nn.Linear(seq_len, seq_len)
        self.mWDN_H.weight = nn.Parameter(self.create_W(seq_len, False))
        self.mWDN_L.weight = nn.Parameter(self.create_W(seq_len, True))
        self.comp_mWDN_H_weight = self.create_W(seq_len, False, is_comp=True).to(device)
        self.comp_mWDN_L_weight = self.create_W(seq_len, True, is_comp=True).to(device)
        self.sigmoid = nn.Sigmoid()
        self.pool = nn.AvgPool1d(2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        hp_1 = self.sigmoid(self.mWDN_H(x))
        lp_1 = self.sigmoid(self.mWDN_L(x))
        hp_out = self.pool(hp_1)
        lp_out = self.pool(lp_1)
        all_out = torch.cat((hp_out, lp_out), dim=-1)
        L_loss = torch.norm((self.mWDN_L.weight.data - self.comp_mWDN_L_weight), 2)
        H_loss = torch.norm((self.mWDN_H.weight.data - self.comp_mWDN_H_weight), 2)
        L_H_loss = self.alpha * L_loss + self.beta * H_loss
        return lp_out, hp_out, all_out, L_H_loss
    
    def create_W(self, P, is_l, is_comp=False):
        if is_l:
            filter_list = self.l_filter
        else:
            filter_list = self.h_filter
        list_len = len(filter_list)
        max_epsilon = np.min(np.abs(filter_list))
        if is_comp:
            weight_np = np.zeros((P, P))
        else:
            weight_np = np.random.randn(P, P) * 0.1 * max_epsilon
        for i in range(0, P):
            filter_index = 0
            for j in range(i, P):
                if filter_index < len(filter_list):
                    weight_np[i][j] = filter_list[filter_index]
                    filter_index += 1
        return _tensor(weight_np)
    
    
def _tensor(x, *rest, **kwargs):
    """Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly."""
    if len(rest): x = (x,) + rest
    # There was a Pytorch bug in dataloader using num_workers>0. Haven't confirmed if fixed
    # if isinstance(x, (tuple,list)) and len(x)==0: return tensor(0)
    res = (x if isinstance(x, Tensor)
           else torch.tensor(x, **kwargs) if isinstance(x, (tuple, list, numbers.Number))
           else _array2tensor(x, **kwargs) if isinstance(x, ndarray)
           else as_tensor(x.values, **kwargs) if isinstance(x, (pd.Series, pd.DataFrame))
           else _array2tensor(array(x), **kwargs))
    if res.dtype is torch.float64: return res.float()
    return res


def _array2tensor(x, requires_grad=False, pin_memory=False, **kwargs):
    if x.dtype == np.uint16: x = x.astype(np.float32)
    # windows default numpy int dtype is int32, while torch tensor default int dtype is int64
    # https://github.com/numpy/numpy/issues/9464
    if sys.platform == "win32" and x.dtype == int: x = x.astype(np.int64)
    t = torch.as_tensor(x, **kwargs)
    t.requires_grad_(requires_grad)
    if pin_memory: t.pin_memory()
    return t

In [47]:
# Construction of multi-level wavelet decomposition network
# 多级小波分解网络构建
class mWDN(nn.Module):
    def __init__(self, c_in, c_out, seq_len, levels=3, wavelet=None,
                 base_arch='gru', rnn_hidden_size=256, rnn_num_layers=1,
                 fcn_hidden_size=512,  alpha=0.5, beta=0.5,
                 get_grad_detail=False, **kwargs):
        super(mWDN, self).__init__()
        self.device = torch.device(torch.cuda.current_device()) if torch.cuda.is_available() else torch.device("cpu")
        self.get_grad_detail = get_grad_detail  # Control whether to output the weight results of the model - 控制是否输出模型的权重结果
        self.base_arch = base_arch  # Selection of backbone network structure - 骨干网络的结构选择
        self.levels = levels  # The level of wavelet decomposition - 小波分解的等级
        self.seq_len = seq_len  # The length of the input sequence - 输入序列的长度
        # Define wavelet decomposition network head and backbone feature extraction network
        # 定义小波分解网络头 和 骨干特征提取网络
        self.mWDN_blocks = nn.ModuleList()
        self.backbone_blocks = nn.ModuleList()
        if base_arch in ['lstm', 'blstm']:
            blstm_or_not = False
            if base_arch == 'blstm':
                blstm_or_not = True
            for i in range(levels):
                self.mWDN_blocks.append(WaveBlock(seq_len // 2 ** i, wavelet=wavelet, alpha=alpha, beta=beta))
                self.backbone_blocks.append(nn.LSTM(c_in - 1, rnn_hidden_size, num_layers=rnn_num_layers,
                                                    batch_first=True, bidirectional=blstm_or_not))  # High frequency features for each wavelet decomposition
            self.backbone_blocks.append(nn.LSTM(c_in - 1, rnn_hidden_size, num_layers=rnn_num_layers,
                                                batch_first=True, bidirectional=blstm_or_not))  # Low frequency features for the last wavelet decomposition
        elif base_arch in ['gru', 'bgru']:
            bgru_or_not = False
            if base_arch == 'bgru':
                bgru_or_not = True
            for i in range(levels):
                self.mWDN_blocks.append(WaveBlock(seq_len // 2 ** i, wavelet=wavelet, alpha=alpha, beta=beta))
                self.backbone_blocks.append(nn.GRU(c_in - 1, rnn_hidden_size, num_layers=rnn_num_layers,
                                                   batch_first=True, bidirectional=bgru_or_not))  # High frequency features for each wavelet decomposition
            self.backbone_blocks.append(nn.GRU(c_in - 1, rnn_hidden_size, num_layers=rnn_num_layers,
                                               batch_first=True, bidirectional=bgru_or_not))  # Low frequency features for the last wavelet decomposition
        if base_arch in ['blstm', 'bgru']:
            rnn_hidden_size = rnn_hidden_size*2

        # Define information fusion network structure
        # 定义信息融合网络结构
        self.output = nn.Linear(rnn_hidden_size * (levels + 1), 256)  # Fully connected integration layer - 全连接整合层
        self.output_2 = nn.Linear(256, c_out)

    def forward(self, x):
        x_l = x[:, :, 1:]  # remove DEPT feature - 去掉最前面的DEPT(深度)特征
        x_list = []
        loss_list = []
        mWDN_dict = {"mWDN_level": [], "x_l": [], "x_h": []}

        for i in range(self.levels):
            x_l, x_h, out_, loss = self.mWDN_blocks[i](x_l)
            loss_list.append(loss)

            x_l = x_l.permute(0, 2, 1)
            x_h = x_h.permute(0, 2, 1)
            mWDN_dict['mWDN_level'].append(i + 1)
            mWDN_dict['x_l'].append(x_l)
            mWDN_dict['x_h'].append(x_h)
            if self.get_grad_detail:
                x_h.requires_grad_(True)  # Preserve gradient - 保留梯度
                x_h.retain_grad()
            if self.base_arch in ['lstm', 'gru', 'blstm', 'bgru']:
                backbone_x_h, _ = self.backbone_blocks[i](x_h)
            else:
                backbone_x_h = self.backbone_blocks[i](x_h)
            x_list.append(backbone_x_h)
            if i == self.levels - 1:
                if self.get_grad_detail:
                    x_l.requires_grad_(True)
                    x_l.retain_grad()
                if self.base_arch in ['lstm', 'gru', 'blstm', 'bgru']:
                    backbone_x_l, _ = self.backbone_blocks[i + 1](x_l)
                else:
                    backbone_x_l = self.backbone_blocks[i](x_h)
                x_list.append(backbone_x_l)
        backbone_all_cat = torch.tensor([]).to(self.device)
        for i in range(len(x_list)):  # Interpolate and integrate the wavelet frequency components decomposed from all features - 将所有特征分解出的小波频率分量进行插值并整合
            x_list[i] = x_list[i].permute(0, 2, 1)
            x_list[i] = F.interpolate(input=x_list[i], size=self.seq_len, mode='linear')
            x_list[i] = x_list[i].permute(0, 2, 1)
            backbone_all_cat = torch.cat((backbone_all_cat, x_list[i]), dim=-1)

        out = backbone_all_cat
        out = self.output(backbone_all_cat)
        out = self.output_2(out)
        loss_mean = torch.stack(loss_list, dim=0).mean(dim=0)
        if self.get_grad_detail:
            return out, loss_mean, mWDN_dict
        else:
            return out, loss_mean

In [48]:
# Model selection
# 模型选择
def build_model(self):
    model = None
    if self.args.model in ['mWDN_lstm']:
        model = mWDN(5, 3, self.args.seq_len, base_arch='lstm',
                     levels=self.args.mWDN_level, wavelet=None,
                     rnn_hidden_size=self.args.mWDN_rnn_hidden_size, rnn_num_layers=self.args.mWDN_rnn_num_layers,
                     fcn_hidden_size=self.args.mWDN_fcn_hidden_size,
                     alpha=self.args.mWDN_alpha, beta=self.args.mWDN_beta)
    elif self.args.model in ['mWDN_gru']:
        model = mWDN(5, 3, self.args.seq_len, base_arch='gru',
                     levels=self.args.mWDN_level, wavelet=None,
                     rnn_hidden_size=self.args.mWDN_rnn_hidden_size, rnn_num_layers=self.args.mWDN_rnn_num_layers,
                     fcn_hidden_size=self.args.mWDN_fcn_hidden_size,
                     alpha=self.args.mWDN_alpha, beta=self.args.mWDN_beta)
    elif self.args.model in ['mWDN_blstm']:
        model = mWDN(5, 3, self.args.seq_len, base_arch='blstm',
                     levels=self.args.mWDN_level, wavelet=None,
                     rnn_hidden_size=self.args.mWDN_rnn_hidden_size, rnn_num_layers=self.args.mWDN_rnn_num_layers,
                     fcn_hidden_size=self.args.mWDN_fcn_hidden_size,
                     alpha=self.args.mWDN_alpha, beta=self.args.mWDN_beta)
    elif self.args.model in ['mWDN_bgru']:
        model = mWDN(5, 3, self.args.seq_len, base_arch='bgru',
                     levels=self.args.mWDN_level, wavelet=None,
                     rnn_hidden_size=self.args.mWDN_rnn_hidden_size, rnn_num_layers=self.args.mWDN_rnn_num_layers,
                     fcn_hidden_size=self.args.mWDN_fcn_hidden_size,
                     alpha=self.args.mWDN_alpha, beta=self.args.mWDN_beta)
    else:
        raise "模型与数据窗口可能不匹配"
    return model
ModelMaker.build_model = build_model

In [49]:
# Instantiation of experiment
# 实验过程的实例化
class Exp_DeepLearning_DTW():
    def __init__(self, args, setting, data_loader, model_maker, mode='train'):
        self.args = args
        self.setting = setting
        self.mode = mode
        # define device - 定义计算设备
        self.device = torch.device("cuda" if args.use_gpu else "cpu")
       
exp = Exp_DeepLearning_DTW(args, setting, train_loader, model_maker, mode='train')

In [50]:
# get model - 
# Note: If you want to train model from the beginning, you need to set the default value of the resume parameter to '' in get_args_parser function
# Search tag "resume-tag"
# 获取模型 - 注意：如果想重新开始模型的训练，需要在前面将resume参数的默认值设置为''.  搜索标记 resume-tag
model = model_maker.build_model().to(exp.device)
if args.resume:
    print('Resuming model ...')
    file_path = os.path.join(args.checkpoints, setting, args.resume)  
    with open(file_path, 'rb') as f:
        model.load_state_dict(torch.load(file_path)) 

Resuming model ...


In [51]:
file_path

'./checkpoints/mWDN_gru-seq(224)-label(224)\\mWDN_gru-0.00596.pth'

### 4.2 Verify model - 验证模型

In [52]:
device = torch.device(torch.cuda.current_device()) if torch.cuda.is_available() else torch.device("cpu")
batch_size = 2
c_in = 5  # Enter the number of features - 输入特征数量
seq_len = 224  # Input sequence length - 输入序列长度
c_out = 3  # Number of output features - 输出特征数量

input = torch.rand(batch_size, seq_len, c_in).to(device).to(torch.float32)  # Define input values - 定义输入值
print(model)

output_pred, _ = model(input)  # forward propagation - 前向传播

mWDN(
  (mWDN_blocks): ModuleList(
    (0): WaveBlock(
      (mWDN_H): Linear(in_features=224, out_features=224, bias=True)
      (mWDN_L): Linear(in_features=224, out_features=224, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (1): WaveBlock(
      (mWDN_H): Linear(in_features=112, out_features=112, bias=True)
      (mWDN_L): Linear(in_features=112, out_features=112, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (2): WaveBlock(
      (mWDN_H): Linear(in_features=56, out_features=56, bias=True)
      (mWDN_L): Linear(in_features=56, out_features=56, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
  )
  (backbone_blocks): ModuleList(
    (0-3): 4 x GRU(4, 256, batch_first=True)
  )
  (output): Linear(in_features=1024, out_features=256, bias=True)
  (output_2): Linear(in_features=256,

In [53]:
output_pred.shape

torch.Size([2, 224, 3])

### 4.3 Optimizer preparation - 优化器准备

In [54]:
# Using the Adam optimizer
# 使用Adam优化器
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=args.lr)

### 4.4 Metric Preparation - 评价指标准备

In [55]:
# Define loss function NMSE
# 定义损失函数NMSE
class NMSELoss(nn.Module):
    def __init__(self):
        super(NMSELoss, self).__init__()

    def forward(self, y_pred, y_true):
        nmse_loss = torch.mean(torch.pow((y_pred - y_true), 2))/torch.mean(torch.pow(y_true, 2))
        return nmse_loss
metrics_nmse = NMSELoss()

In [56]:
# Define loss function MAD
# 定义损失函数MAD
class MADLoss(nn.Module):
    def __init__(self):
        super(MADLoss, self).__init__()

    def forward(self, y_pred, y_true):
        mad_loss = torch.mean(torch.abs(y_pred - y_true))
        return mad_loss
metrics_mad = MADLoss()

### 4.5 Training process and validation process - 训练过程及验证过程

In [57]:
# instantiating a timer object
timer = Timer()

In [58]:
# Using the training strategy of EarlyStopping, the training stops when the accuracy of the validation set has not been improved for several consecutive rounds
# 使用EarlyStopping的训练策略，当验证集精度连续几轮未提升时训练停止
from tqdm import tqdm
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):  
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 0
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        
    def __call__(self, val_loss, model, path, epoch):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0
        return self.counter

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path+'/'+'checkpoint.pth')
        self.val_loss_min = val_loss

In [59]:
# Reverse normalization of neural network predictions
# 对神经网络的预测进行反归一化
def deeplearning_predict(data, label, model):
    if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
        predictions, L_H_loss = model(data.to(exp.device))
    else:
        predictions = model(data.to(exp.device))
    # Complete the reverse normalization of neural network predictions
    # 完成对神经网络的预测的反归一化
    np_pred = scaler.inverse_transform(predictions,
                                            train_df_mean[5:], train_df_std[5:],
                                            train_df_max[5:], train_df_min[5:],
                                            mode=args.scaler)
    pred_label = torch.tensor(np_pred, dtype=torch.float32)
    # Use denormalization/normalization to approximate the true value or consider using idx to search for the true value in the future (however, this is in the form of (128,10,6), which is difficult to use in the form of dataframe)
    # 使用反归一化/标准化逼近真实值  或者  以后考虑使用idx搜到真实值(不过这里是(128,10,6)的形式, 使用dataframe的形式比较困难)
    np_label = scaler.inverse_transform(label,
                                                train_df_mean[5:], train_df_std[5:],
                                                train_df_max[5:], train_df_min[5:],
                                                mode=args.scaler)
    ture_label = torch.tensor(np_label, dtype=torch.float32)
    return pred_label, ture_label

In [60]:
# training model - 训练模型
path = os.path.join(args.checkpoints, setting)  # Define the best model storage location - 定义最佳模型保存地点
dir_exist(path)
early_stopping = EarlyStopping(patience=args.patience, verbose=True)   # Define early stop - 定义提前停止
nmseloss_list = []
# At any point you can hit Ctrl + C to break out of training early.
if args.resume:
    print("Reload existing models and no longer training - 读取已有的模型，不再训练")
else:
    try:
        # Training process|
        for epoch in range(args.train_epochs):
            train_loss = []
            model.train()
            for data, ture_label, idx in tqdm(train_loader, total=len(train_loader), desc='Train:'):
                data, ture_label = data.to(exp.device), ture_label.to(exp.device)
                if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
                    pred, L_H_loss = model(data)  # forward propagation
                    loss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3]) + L_H_loss * 0.01
                else:
                    pred = model(data)  # forward propagation
                    loss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3])
                assert pred.shape[2] == 3, "模型使用错误"

                train_loss.append(loss.item())  # record loss
                optimizer.zero_grad()  # Perform gradient reset
                loss.backward()  # Back Propagation
                model_optim.step()  # Optimize and update model parameters
            train_loss = np.average(train_loss)

            # Verification process - train_nmse_loss
            # 验证过程 - train_nmse_loss
            model.eval()
            total_nmseloss = []
            with torch.no_grad():
                for data, ture_label, idx in tqdm(train_loader, total=len(train_loader), desc='Validation:'):
                    data, ture_label = data.to(exp.device), ture_label.to(exp.device)
                    if not args.scale:
                        if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
                            pred, L_H_loss = model(data)
                        else:
                            pred = model(data)
                    else:
                        pred, ture_label = deeplearning_predict(data, ture_label, model)

                    nmseloss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3])
                    total_nmseloss.append(nmseloss.item())
            train_nmse_loss = np.average(total_nmseloss)
            
            # Verification process - val_nmse_loss
            # 验证过程 - val_nmse_loss
            model.eval()
            total_nmseloss = []
            with torch.no_grad():
                for data, ture_label, idx in tqdm(val_loader, total=len(val_loader), desc='Validation:'):
                    data, ture_label = data.to(exp.device), ture_label.to(exp.device)
                    if not args.scale:
                        if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
                            pred, L_H_loss = model(data)
                        else:
                            pred = model(data)
                    else:
                        pred, ture_label = deeplearning_predict(data, ture_label, model)
                        
                    nmseloss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3])
                    total_nmseloss.append(nmseloss.item())
            val_nmse_loss = np.average(total_nmseloss)

            nmseloss_list.append([epoch + 1, train_nmse_loss, val_nmse_loss])

            pprint(f"Epoch: {epoch + 1}, Steps: {len(train_loader)}, Train Loss: {train_loss:.5f}, "
                        f"Train NMSELoss: {train_nmse_loss:.5f}, Validation NMSELoss: {val_nmse_loss:.5f}")

            early_stopping(val_nmse_loss, model, path, epoch)
            args.end_epoch_num = epoch + 1
            

            if early_stopping.early_stop:
                pprint("Early stopping and best val score{} @ epoch{}".format(
                    early_stopping.best_score, early_stopping.best_epoch + 1))
                break
    except KeyboardInterrupt:
        pprint('-' * 89)
        pprint('******Exiting from training early******')

Reload existing models and no longer training - 读取已有的模型，不再训练


In [61]:
# Training completed, stop timing
# 训练完毕，停止计时
args.train_val_time = timer.stop()
pprint(args.log_file, f'Time spending on training and evaluation: {args.train_val_time:.6f} sec')

[2023-09-15 16:58:05] - Time spending on training and evaluation: 0.065998 sec


In [62]:
# Training process testing
pprint(args.log_file, '>>>>>>>Testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
mode = "train" 
model.eval()
total_nmseloss = []
with torch.no_grad():
    for data, ture_label, idx in tqdm(train_loader, total=len(train_loader), desc='Validation: trainset'):
        data, ture_label = data.to(exp.device), ture_label.to(exp.device)
        pred, ture_label = deeplearning_predict(data, ture_label, model)
           
        nmseloss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3])
        total_nmseloss.append(nmseloss.item())
train_nmse_loss = np.average(total_nmseloss)

[2023-09-15 16:58:05] - >>>>>>>Testing : mWDN_gru-seq(224)-label(224)<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


Validation: trainset: 100%|██████████| 1177/1177 [00:36<00:00, 32.09it/s]


In [63]:
# Verification process testing
mode = "val"
model.eval()
total_nmseloss = []
with torch.no_grad():
    for data, ture_label, idx in tqdm(val_loader, total=len(val_loader), desc='Validation: valset'):
        data, ture_label = data.to(exp.device), ture_label.to(exp.device)
        pred, ture_label = deeplearning_predict(data, ture_label, model)

        nmseloss = metrics_nmse(pred[:, :, :], ture_label[:, :, :3])
        total_nmseloss.append(nmseloss.item())
val_nmse_loss = np.average(total_nmseloss)

Validation: valset: 100%|██████████| 399/399 [00:11<00:00, 33.95it/s]


In [64]:
# Save the best model and argparse namespace, and output NMSE and MAD values
# 保存最佳模型及argparse命名空间，并输出NMSE和MAD值
pprint(args.log_file, f"Train NMSELoss: {train_nmse_loss:.5f}, Validation NMSELoss: {val_nmse_loss:.5f}")
# Save Historical Best Model
# 保存历史最佳模型
torch.save(model.state_dict(), os.path.join(args.checkpoints, setting)
            + '/' + f'{args.model}-{val_nmse_loss:.5f}.pth')
# Save the current argparse namespace to the JSON file
# 保存目前的argparse命名空间到json文件中
argparse_json(args, setting, mode='save', info=f'{val_nmse_loss:.5f}')

[2023-09-15 16:58:53] - Train NMSELoss: 0.00252, Validation NMSELoss: 0.00543


### 4.6 DTW
>Dynamic Time Warping(DTW) is a well-known technique used to determine alignment between two temporal sequences which be introduced in 1960s(Vintsyuk, 1968). At present, DTW has been widely applied in fields such as speech recognition, biological sequence similarity analysis, and human motion recognition. DTW constructs a corresponding relationship between two sequence elements based on the principle of closest distance, and aligns a distorted, stretched, or compressed time series. By calculating the distance value and cumulative distance value, backtracking to find the shortest path in the cumulative distance matrix.

>However, calculating the degree of offset through distance can lead to an erroneous one-to-many situation, where a single point on one time series maps to a portion of data on another time series, which is called singularity. Eamon et al(2001) introduces Derivative DTW(DDTW) to alleviate singularity problems.


In [65]:
pprint(args.log_file, '>>>>>>>Predicting with DTW : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))

[2023-09-15 16:58:53] - >>>>>>>Predicting with DTW : mWDN_gru-seq(224)-label(224)<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [66]:
# import libraries
import tsaug as tsaug
from dtwalign import dtw as dtwa
from scipy._lib._util import _asarray_validated, float_factorial
from numpy import (array, transpose, searchsorted, atleast_1d, atleast_2d,
                   ravel, poly1d, asarray, intp)
from scipy.interpolate import interp1d

In [67]:
# Obtain DTW path in the form of a one-dimensional array
# 以一维的数组的形式获得DTW路径
def get_warping_path(paths, target="query"):
    if target not in ("query", "reference"):
        raise ValueError("target argument must be 'query' or 'reference'")
    if target == "reference":
        xp = pd.DataFrame(paths).iloc[:, 0]
        zero_x = xp[xp.values == 0].shape[0] - 1
        xp = pd.DataFrame(paths).iloc[zero_x:, 0]  # query path
        yp = pd.DataFrame(paths).iloc[zero_x:, 1]  # reference path
    else:
        xp = pd.DataFrame(paths).iloc[:, 1]
        zero_x = xp[xp.values == 0].shape[0] - 1
        yp = pd.DataFrame(paths).iloc[zero_x:, 0]  # query path
        xp = pd.DataFrame(paths).iloc[zero_x:, 1]  # reference path
    interp_func = interp1d(xp, yp, kind="linear")  # define the independent and dependent variables of a function - 首先定义一个函数的自变量和因变量
    # get warping index as float values and then convert to int
    # note: Ideally, the warped value should be calculated as mean.
    #       (in this implementation, just use value corresponds to rounded-up index)
    # Given a sequence of x, based on the above function, return the sequence value of y in the function - 给一个x的序列，根据上述的函数，返回y在函数中的序列值
    warping_index = interp_func(np.arange(xp.min(), xp.max() + 1)).astype(np.int64)
    # the most left side gives nan, so substitute first index of path
    warping_index[0] = yp.min()
    return warping_index

In [68]:
# This function is used for the calculation of DDTW, converting sequence data into derivative estimates - 该函数用于DDTW的计算，将序列数据转换成导数估计值
def __derivation(ts, ):
    drts = []
    for i in range(ts.index[0]+1, ts.index[-1]):
        derived = ((ts[i] - ts[i - 1]) + ((ts[i + 1] - ts[i - 1]) / 2.0)) / 2.0
        drts.append(derived)
    return drts

In [69]:
# Obtain aligned data
# 获得对齐后数据
def align_dtw(data, method='dtwalign', dis_function=None, window_size=180, window_type="sakoechiba",
              step_pattern="symmetricP05"):
    for feature in ['RHOB', 'NPHI', 'log_RD']:
        # Predict alignment data - 预测对齐数据
        a1 = data[f'{feature}']
        a2 = data[f'{feature}_reg']

        x1 = a1.copy()
        x1 = x1.to_numpy()
        xc1 = tsaug.Convolve(window='triang', size=7).augment(x1)
        s1 = pd.Series(xc1, name=f'{feature}')
        x2 = a2.copy()
        x2 = x2.to_numpy()
        xc2 = tsaug.Convolve(window='triang', size=7).augment(x2)
        s2 = pd.Series(xc2, name=f'{feature}_reg')
        if method == 'dtwalign':
            res = dtwa(s1, s2, window_type=window_type, window_size=window_size, step_pattern=step_pattern,
                       dist_only=False, open_begin=False, open_end=False)
            paths, distance, path_matrix = res.path, res.distance, res.cumsum_matrix
            path_query2ref = get_warping_path(paths, target="reference")
            path_ref2query = get_warping_path(paths, target="query")
        elif method == 'dtwalign_ddtw': 
            # Convert the sequence data into its derivative estimate before conducting DTW - 在进行dtw之前先将序列数据转换成其导数估计值
            s1 = pd.Series(__derivation(s1), name=f'{feature}')
            s2 = pd.Series(__derivation(s2), name=f'{feature}_reg')
            res = dtwa(s1, s2, window_size=window_size, window_type=window_type, step_pattern=step_pattern,
                       dist_only=False, open_begin=False, open_end=False)
            paths = res.path
            # align index in paths - paths前后补齐对齐索引
            paths = (paths + 1)
            paths = np.row_stack((np.array([0, 0]), paths))
            paths = np.row_stack((paths, np.array([s1.shape[0]+1, s2.shape[0]+1])))
            path_query2ref = get_warping_path(paths, target="reference")
            path_ref2query = get_warping_path(paths, target="query")
        else:
            raise 'method is error, method: "dtaidistance", "dtwalign"'

        data.insert(data.shape[1], column=f'{feature}_dept_pred_dtw', value=data['DEPT'].values[path_query2ref])
        dup_d = data[f'{feature}_dept_pred_dtw'].duplicated()
        data.loc[dup_d, f'{feature}_dept_pred_dtw'] = None

        data.insert(data.shape[1], column=f'{feature}_pred_dtw', value=data[f'{feature}'].values[path_ref2query])
        dup = data[f'{feature}_pred_dtw'].duplicated()
        data.loc[dup, f'{feature}_pred_dtw'] = None
        data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
        data.insert(data.shape[1], column=f'{feature}_path_query2ref_dtw', value=path_query2ref)
        data.insert(data.shape[1], column=f'{feature}_path_ref2query_dtw', value=path_ref2query)

    columns_list = ['wellnum', 'DEPT', 'GR', 'RHOB', 'NPHI', 'log_RD', 'RHOB_pred', 'NPHI_pred', 'log_RD_pred',
                    'RHOB_dept_pred', 'NPHI_dept_pred', 'log_RD_dept_pred',
                    'RHOB_reg', 'NPHI_reg', 'log_RD_reg',
                    'RHOB_path_query2ref_dtw', 'NPHI_path_query2ref_dtw', 'log_RD_path_query2ref_dtw',
                    'RHOB_path_ref2query_dtw', 'NPHI_path_ref2query_dtw', 'log_RD_path_ref2query_dtw',
                    'RHOB_pred_dtw', 'NPHI_pred_dtw', 'log_RD_pred_dtw',
                    'RHOB_dept_pred_dtw', 'NPHI_dept_pred_dtw', 'log_RD_dept_pred_dtw']
    data = data[columns_list]
    return data

In [70]:
# Save the DTW results and merge them with the original data
def DTW(data_all, args=None, mode='training'):
    current_path = "./results"  # Obtain the absolute path of the current file - 获取当前文件的绝对路径
    dtw_path = os.path.join(current_path, 'dtw_results', mode)  # Path for storing DTW predicted values - dtw预测值存的路径
    dir_exist(dtw_path)
    pred_y = pd.DataFrame()
    for wellnum in tqdm(data_all['wellnum'].unique()):
        data_dtw = data_all[data_all['wellnum'] == wellnum]
        if not args:
            pass

        df_data = align_dtw(data_dtw, method=args.dtw_method, dis_function=None, window_size=args.dtw_window_size,
                            window_type=args.dtw_window_type, step_pattern=args.dtw_step_pattern)
        pred_y_temp = df_data.loc[:, 'RHOB_pred_dtw':]
        pred_y = pd.concat([pred_y, pred_y_temp], axis=0)
        if mode != 'training':
            dir_exist(dtw_path)
            df_data.to_csv(os.path.join(dtw_path, f"dtw_well_{wellnum}.csv"), index=False)
    return pred_y

In [71]:
# Training Set Prediction
# 训练集预测 
mode = "train"
pred_list = []
for data, ture_label, idx in tqdm(train_loader, total=len(train_loader)):
    data, ture_label = data.to(exp.device), ture_label.to(exp.device)
    if not args.scale:
        if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
            pred, _ = model(data)
        else:
            pred = model(data)
    else:
        pred, ture_label = deeplearning_predict(data, ture_label, model)
    for batch_idx in range(data.shape[0]):
        df_pred = pd.DataFrame(pred.cpu().detach().numpy()[batch_idx, :, :],
                                index=idx.detach().numpy()[batch_idx, :, 0],
                                columns=args.label_columns[:3])
        pred_list.append(df_pred)
df_predict = pd.concat(pred_list)
df_predict.sort_index(ascending=True, inplace=True, axis=0, key=None)
# Deduplication based on index, or taking the optimal value
# 根据index进行去重，或者说取最优数值
df_predict = df_predict.iloc[:, :3]
df_predict = df_predict.groupby(df_predict.index).median()
df_predict.rename(columns={'RHOB_pred': 'RHOB_reg', 'NPHI_pred': 'NPHI_reg', 'log_RD_pred': 'log_RD_reg'},
                    inplace=True)

df_all = train_df
df_dtw = df_all.join(df_predict, how='left', sort=False)
dir_exist(args.log_file)
       

100%|██████████| 1177/1177 [00:54<00:00, 21.57it/s]


In [72]:
 # DTW alignment of data
 # 对数据进行DTW对齐
df_pred_y = DTW(df_dtw, args=args, mode=mode)

  0%|          | 0/21 [00:00<?, ?it/s]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFra

In [73]:
# Calculate the loss function and save the prediction results
# 计算损失函数并保存预测结果
df_predict = pd.DataFrame(df_pred_y.values, columns=args.label_columns)
if mode != 'test':
    # dataframe->array->tensor
    tensor_label = torch.tensor(df_all[args.label_columns].values, dtype=torch.float32).unsqueeze(0)
    # tensor(200, 6) -> tensor(1, 200, 6)
    tensor_predict = torch.tensor(df_predict.values, dtype=torch.float32).unsqueeze(0)
    nmseloss = metrics_nmse(tensor_predict[:, :, :3], tensor_label[:, :, :3])
    madloss = metrics_mad(tensor_predict[:, :, 3:], tensor_label[:, :, 3:])
    pprint(args.log_file, f"{mode}-results after DTW: NMSELoss: {nmseloss:.5f}, MADLoss: {madloss:.5f}") 
df_all = df_all.reset_index().drop(columns=['index'], axis=1)  # 先重置索引，后删除生成的原始备份index列
df_all[args.label_columns] = df_predict[args.label_columns]
df_all['log_RD'] = 10 ** df_all['log_RD']
df_all['log_RD_pred'] = 10 ** df_all['log_RD_pred']
df_all.rename(columns={'log_RD': 'RD', 'log_RD_pred': 'RD_pred', 'log_RD_dept_pred': 'RD_dept_pred'},
                inplace=True)
# save predict results
output_path = os.path.join(r'results/predict/', args.model, mode)
dir_exist(output_path)
wellnum_list = df_all['wellnum'].unique()
for wellnum in wellnum_list:
    df = df_all[df_all['wellnum'] == wellnum].copy()
    df = df.drop('wellnum', axis=1)
    df.to_csv(os.path.join(output_path, f'aligned_well_{wellnum}.csv'),
                index=False, encoding='utf_8_sig')

[2023-09-15 17:01:17] - train-results after DTW: NMSELoss: 0.00218, MADLoss: 5.88079


In [74]:
# Validate set prediction, calculate loss function, and save results
# 验证集预测、计算损失函数并保存结果
mode = "val"
pred_list = []
for data, ture_label, idx in tqdm(val_loader, total=len(val_loader)):
    data, ture_label = data.to(exp.device), ture_label.to(exp.device)
    if not args.scale:
        if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
            pred, _ = model(data)
        else:
            pred = model(data)
    else:
        pred, ture_label = deeplearning_predict(data, ture_label, model)
    for batch_idx in range(data.shape[0]):
        df_pred = pd.DataFrame(pred.cpu().detach().numpy()[batch_idx, :, :],
                                index=idx.detach().numpy()[batch_idx, :, 0],
                                columns=args.label_columns[:3])
        pred_list.append(df_pred)
df_predict = pd.concat(pred_list)
df_predict.sort_index(ascending=True, inplace=True, axis=0, key=None)
# Deduplication based on index, or taking the optimal value
# 根据index进行去重，或者说取最优数值
df_predict = df_predict.iloc[:, :3]
df_predict = df_predict.groupby(df_predict.index).median()
df_predict.rename(columns={'RHOB_pred': 'RHOB_reg', 'NPHI_pred': 'NPHI_reg', 'log_RD_pred': 'log_RD_reg'},
                    inplace=True)

df_all = val_df
df_dtw = df_all.join(df_predict, how='left', sort=False)
 # DTW
df_pred_y = DTW(df_dtw, args=args, mode=mode)
df_predict = pd.DataFrame(df_pred_y.values, columns=args.label_columns)
if mode != 'test':
    # dataframe->array->tensor
    tensor_label = torch.tensor(df_all[args.label_columns].values, dtype=torch.float32).unsqueeze(0)
    # tensor(200, 6) -> tensor(1, 200, 6)
    tensor_predict = torch.tensor(df_predict.values, dtype=torch.float32).unsqueeze(0)
    nmseloss = metrics_nmse(tensor_predict[:, :, :3], tensor_label[:, :, :3])
    madloss = metrics_mad(tensor_predict[:, :, 3:], tensor_label[:, :, 3:])
    pprint(args.log_file, f"{mode}-results after DTW: NMSELoss: {nmseloss:.5f}, MADLoss: {madloss:.5f}") 
df_all = df_all.reset_index().drop(columns=['index'], axis=1)  # Reset the index first, then delete the generated original backup index column - 先重置索引，后删除生成的原始备份index列
df_all[args.label_columns] = df_predict[args.label_columns]
df_all['log_RD'] = 10 ** df_all['log_RD']
df_all['log_RD_pred'] = 10 ** df_all['log_RD_pred']
df_all.rename(columns={'log_RD': 'RD', 'log_RD_pred': 'RD_pred', 'log_RD_dept_pred': 'RD_dept_pred'},
                inplace=True)
# save predict results
output_path = os.path.join(r'results/predict/', args.model, mode)
dir_exist(output_path)
wellnum_list = df_all['wellnum'].unique()
for wellnum in wellnum_list:
    df = df_all[df_all['wellnum'] == wellnum].copy()
    df = df.drop('wellnum', axis=1)
    df.to_csv(os.path.join(output_path, f'aligned_well_{wellnum}.csv'),
                index=False, encoding='utf_8_sig')
if mode in ['test']:
    shutil.make_archive(os.path.join(r'results/predict/', args.model, 'dreamstar_submission_1'),
                        'zip', output_path)

  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\260

[2023-09-15 17:02:07] - val-results after DTW: NMSELoss: 0.00426, MADLoss: 9.78042


In [75]:
# Release currently held and unoccupied cache graphics memory
# 释放当前持有且未被占用的缓存显存
torch.cuda.empty_cache()

In [76]:
pprint(args.log_file, '>>>>>>>End Training model: {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))

[2023-09-15 17:02:08] - >>>>>>>End Training model: mWDN_gru-seq(224)-label(224)<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


## 5. model prediction - 模型预测

### 5.1 Reading Test set data - 测试集数据读取 

In [77]:
# Obtain test set data - 获得测试集数据
current_path = os.getcwd()  # absolute path of the current file - 获取当前文件的绝对路径
data_path = os.path.join(current_path, 'data/test')  # Obtain the path of the data file - 获取数据文件的路径
files = glob.glob(f'{data_path}//aligned_well_0*.csv')
df = []
y_feat = ['RHOB', 'NPHI', 'log_RD']
for i in files:
    df0 = pd.read_csv(i)
    df0.rename(columns={'RD_pred': 'log_RD_pred'}, inplace=True)
    df0.rename(columns={'RD_dept_pred': 'log_RD_dept_pred'}, inplace=True)
    df0['log_RD'] = np.log10(df0['RD'])
    df0.drop('RD', axis=1, inplace=True)
    # for feat in y_feat:
    #     df0[f'{feat}_pred'] = df0[f'{feat}'].copy()
    # for feat in y_feat:  # Two cycles to ensure order - 两次循环是为了保证顺序
    #     df0[f'{feat}_dept_pred'] = df0['DEPT'].copy()
    df0['wellnum'] = i.split('_')[-1].split('.')[0]
    df.append(df0.copy())
test_df = pd.concat(df)

In [78]:
# Take the logarithm of RD - 将RD取对数
# Print Test Set DataFrame
# 打印测试集DataFrame
test_df

,DEPT,GR,RHOB,NPHI,RHOB_pred,NPHI_pred,log_RD_pred,RHOB_dept_pred,NPHI_dept_pred,log_RD_dept_pred,log_RD,wellnum
0,1808.0,55.1473,2.275160,0.343836,NaN,NaN,NaN,NaN,NaN,NaN,0.754111,01
1,1808.5,54.2181,2.000059,0.332846,NaN,NaN,NaN,NaN,NaN,NaN,2.761481,01
2,1809.0,53.2889,1.999823,0.323871,NaN,NaN,NaN,NaN,NaN,NaN,3.232734,01
3,1809.5,53.0682,2.000078,0.324658,NaN,NaN,NaN,NaN,NaN,NaN,2.945427,01
4,1810.0,58.5734,1.999922,0.312027,NaN,NaN,NaN,NaN,NaN,NaN,2.508181,01
...,...,...,...,...,...,...,...,...,...,...,...,...
3184,5433.5,19.5709,2.698139,0.060279,NaN,NaN,NaN,NaN,NaN,NaN,2.995679,03
3185,5434.0,19.5638,2.667668,0.060544,NaN,NaN,NaN,NaN,NaN,NaN,2.866213,03
3186,5434.5,19.5581,2.665155,0.060932,NaN,NaN,NaN,NaN,NaN,NaN,2.738212,03
3187,5435.0,19.8559,2.660236,0.060864,NaN,NaN,NaN,NaN,NaN,NaN,2.623339,03


### 5.2  Normalization and segmentation of test set data - 测试集数据归一化和序列切分

In [79]:
df_raw = test_df

In [80]:
# data normalization - 数据归一化
cols = list(df_raw.columns)
for col in label_columns:  # Remove target column - 去掉预测目标列
    cols.remove(col)
cols.remove('wellnum')  # Remove wellnum column - 去掉井名
df_raw = df_raw[['wellnum'] + cols + [x for x in label_columns]]  # Reorder columns-对列重新排序
cols_list = df_raw.columns[1:]  # Other columns names without wellnum - 不取井名 - 为了取归一化/标准化参数
df_data = df_raw[cols_list]

In [81]:
# Normalize the test set using the existing mean, variance, maximum, and minimum values of the training set
# 测试集归一化使用训练集已有的均值、方差、最大值、最小值
test_df_mean, test_df_std = pd.Series(args.train_data_mean), pd.Series(args.train_data_std)
test_df_max, test_df_min = pd.Series(args.train_data_max), pd.Series(args.train_data_min)

In [82]:
print("test_df_mean:",test_df_mean)
print("test_df_std:",test_df_std)
print("test_df_max:",test_df_max)
print("test_df_min:",test_df_min)

test_df_mean: DEPT                2823.193432
GR                   105.928636
RHOB                   2.485743
NPHI                   0.267164
log_RD                 0.911849
RHOB_pred              2.485289
NPHI_pred              0.267315
log_RD_pred            0.910886
RHOB_dept_pred      2827.711509
NPHI_dept_pred      2825.248832
log_RD_dept_pred    2826.225368
dtype: float64
test_df_std: DEPT                1304.769960
GR                    30.254012
RHOB                   0.131851
NPHI                   0.077829
log_RD                 0.391379
RHOB_pred              0.132835
NPHI_pred              0.077794
log_RD_pred            0.391397
RHOB_dept_pred      1305.584699
NPHI_dept_pred      1305.427222
log_RD_dept_pred    1305.205395
dtype: float64
test_df_max: DEPT                5743.000000
GR                   400.000000
RHOB                   3.045315
NPHI                   0.665037
log_RD                 4.697986
RHOB_pred              3.045400
NPHI_pred              0.665000
lo

In [83]:
# Obtain data from different wells - 获取不同井的数据
df_data_well_list = []
for wellnum in df_raw['wellnum'].unique():
    df_temp = df_raw[df_raw['wellnum'] == wellnum]
    df_data_well_list.append(df_temp)   

In [84]:
# Sequential segmentation of test set features and labels - 对测试集特征和标签进行序列切分
def make_test_feat_seq_list(seq_list, all_len, feat, np_list, start_index, end_index, seq_len):
    for i in range(all_len):
        np_temp = feat.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[0:seq_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

def make_test_label_seq_list(seq_list, all_len, label, np_list, start_index, end_index, shift_len, pred_len):
    for i in range(all_len):
        np_temp = label.loc[start_index + i: end_index - all_len + i + 1].to_numpy()
        np_list.append(np_temp)
    np_list_stack = np_list[shift_len:shift_len + pred_len]
    np_seq = np.stack(np_list_stack, axis=1)
    seq_list.append(np_seq)
    return seq_list

In [85]:
# Standardize the test data and obtain the features, labels, and indexes of the test data
# 首先对测试数据进行标准化，之后获得测试数据的特征、标签、和index
test_feat_seq_list = []
test_label_seq_list = []
test_index_seq_list = []
for df_data in df_data_well_list:
    if scale:
        df_data = scaler.transform(df_data[cols + [x for x in label_columns]],
                                        test_df_mean, test_df_std,
                                        test_df_max, test_df_min, mode=args.scaler)
    df_data['index'] = df_data.index
    start_index = df_data.index[0]
    end_index = df_data.index[-1]
    feat, label, index = df_data[cols], df_data[label_columns], df_data[['index']]
    # make sequence data - 构造序列数据
    feat_list = []
    label_list = []
    index_list = []
    if seq_len < shift_len + pred_len:
        all_len = shift_len + pred_len
    else:
        all_len = seq_len
    test_feat_seq_list = make_test_feat_seq_list(test_feat_seq_list, all_len, feat, feat_list,
                                            start_index, end_index, seq_len)
    test_label_seq_list = make_test_label_seq_list(test_label_seq_list, all_len, label, label_list,
                                                start_index, end_index, shift_len, pred_len)
    test_index_seq_list = make_test_feat_seq_list(test_index_seq_list, all_len, index, index_list,
                                                start_index, end_index, seq_len)

# Data connection between different wells - 不同井之间的数据相连
test_feat_seq_all = np.concatenate(test_feat_seq_list, axis=0)
test_label_seq_all = np.concatenate(test_label_seq_list, axis=0)
test_index_seq_all = np.concatenate(test_index_seq_list, axis=0)

In [86]:
# The segmented sequence data form, where 224 represents the sequence length, 5 represents the dimension of input data features, and 6 represents the feature dimension of the predicted target
# 切分后的序列数据形式，224代表序列长度，5代表输入数据特征的维度，6代表预测目标的特征维度
test_feat_seq_all[0].shape, test_label_seq_all[0].shape, test_index_seq_all[0].shape

((224, 5), (224, 6), (224, 1))

### 5.3 Dataset and dataloader in test set - 测试集中的Dataset 和dataloader

In [87]:
# Prepare test data adapted to neural networks through DataLoader
# 通过DataLoader准备与神经网络适配的test数据
test_dataset = WellDataset(test_feat_seq_all, test_label_seq_all, test_index_seq_all)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)

In [88]:
# test test_loader
# 测试一下test_loader
next(iter(test_loader))

[tensor([[[0.4297, 0.1864, 0.7205, 0.3899, 0.2758],
          [0.4298, 0.1877, 0.7194, 0.3889, 0.2730],
          [0.4299, 0.1872, 0.7194, 0.3880, 0.2720],
          ...,
          [0.4504, 0.2119, 0.7257, 0.3471, 0.2820],
          [0.4504, 0.2217, 0.7240, 0.3450, 0.2819],
          [0.4505, 0.2206, 0.7224, 0.3428, 0.2806]],
 
         [[0.7320, 0.1055, 0.7000, 0.1654, 0.2770],
          [0.7320, 0.1104, 0.6992, 0.1795, 0.2756],
          [0.7321, 0.1373, 0.6975, 0.1957, 0.2757],
          ...,
          [0.7526, 0.2373, 0.7073, 0.2936, 0.2619],
          [0.7527, 0.2398, 0.7036, 0.2874, 0.2603],
          [0.7528, 0.2333, 0.7028, 0.2672, 0.2590]],
 
         [[0.0706, 0.2329, 0.6375, 0.4358, 0.2558],
          [0.0707, 0.2344, 0.6365, 0.4370, 0.2559],
          [0.0708, 0.2336, 0.6366, 0.4378, 0.2567],
          ...,
          [0.0912, 0.2459, 0.6952, 0.4841, 0.2595],
          [0.0913, 0.2374, 0.6951, 0.4744, 0.2583],
          [0.0914, 0.2319, 0.6949, 0.4649, 0.2560]],
 
         .

### 5.4 Overload model and JSON file - 模型重载和json文件的重载

In [89]:
# Model overload - 模型重载
checkpoints_path = r'./checkpoints\mWDN_gru-seq(224)-label(224)\mWDN_gru-0.00596.pth'
model.load_state_dict(torch.load(checkpoints_path))
# JSON file overload - json文件重载
file = os.path.join(args.checkpoints, setting, "json", "mWDN_gru-0.00543.json")  
with open(file,"r") as f:
    args = json.load(f)

In [90]:
print(args)

{'data_path': './data', 'checkpoints': './checkpoints/', 'seq_len': 224, 'label_len': 224, 'shift_len': 0, 'train_columns': ['DEPT', 'GR', 'RHOB', 'NPHI', 'log_RD'], 'label_columns': ['RHOB_pred', 'NPHI_pred', 'log_RD_pred', 'RHOB_dept_pred', 'NPHI_dept_pred', 'log_RD_dept_pred'], 'data_augment_num': 3, 'scale': True, 'scaler': 'Normalization', 'train_data_mean': {'DEPT': 2823.193431925245, 'GR': 105.92863588501042, 'RHOB': 2.485743085562526, 'NPHI': 0.2671636977614579, 'log_RD': 0.9118489593550839, 'RHOB_pred': 2.485289282956213, 'NPHI_pred': 0.2673148158158931, 'log_RD_pred': 0.9108855469613067, 'RHOB_dept_pred': 2827.711509254254, 'NPHI_dept_pred': 2825.2488319561357, 'log_RD_dept_pred': 2826.2253681107936}, 'train_data_std': {'DEPT': 1304.7699596687644, 'GR': 30.25401193390226, 'RHOB': 0.13185065904918125, 'NPHI': 0.0778286033404489, 'log_RD': 0.39137861036047716, 'RHOB_pred': 0.13283539077198997, 'NPHI_pred': 0.07779364757549828, 'log_RD_pred': 0.3913965358652954, 'RHOB_dept_pred'

In [91]:
# Namespace conversion
# 命名空间转换
args = argparse.Namespace(**args)
print(args)

Namespace(data_path='./data', checkpoints='./checkpoints/', seq_len=224, label_len=224, shift_len=0, train_columns=['DEPT', 'GR', 'RHOB', 'NPHI', 'log_RD'], label_columns=['RHOB_pred', 'NPHI_pred', 'log_RD_pred', 'RHOB_dept_pred', 'NPHI_dept_pred', 'log_RD_dept_pred'], data_augment_num=3, scale=True, scaler='Normalization', train_data_mean={'DEPT': 2823.193431925245, 'GR': 105.92863588501042, 'RHOB': 2.485743085562526, 'NPHI': 0.2671636977614579, 'log_RD': 0.9118489593550839, 'RHOB_pred': 2.485289282956213, 'NPHI_pred': 0.2673148158158931, 'log_RD_pred': 0.9108855469613067, 'RHOB_dept_pred': 2827.711509254254, 'NPHI_dept_pred': 2825.2488319561357, 'log_RD_dept_pred': 2826.2253681107936}, train_data_std={'DEPT': 1304.7699596687644, 'GR': 30.25401193390226, 'RHOB': 0.13185065904918125, 'NPHI': 0.0778286033404489, 'log_RD': 0.39137861036047716, 'RHOB_pred': 0.13283539077198997, 'NPHI_pred': 0.07779364757549828, 'log_RD_pred': 0.3913965358652954, 'RHOB_dept_pred': 1305.5846989649228, 'NPHI

### 5.5  Model prediction - 模型预测

In [92]:
# Define the model and model file names that need to be overloaded
# 定义需要重载的模型和模型文件名称

args.resume = 'mWDN_gru-0.00596.pth'  # Finall - NMSE-best
args.model = 'mWDN_gru'

In [93]:
# Test set prediction, calculation of loss function, and saving of results
# 测试集预测、计算损失函数并保存结果
mode = "test"
pred_list = []
for data, ture_label, idx in tqdm(test_loader, total=len(test_loader)):
    data, ture_label = data.to(exp.device), ture_label.to(exp.device)
    if not args.scale:
        if args.model in ['wavelet_lstm', 'mWDN_lstm', 'mWDN_gru', 'mWDN_blstm', 'mWDN_bgru']:
            pred, _ = model(data)
        else:
            pred = model(data)
    else:
        pred, ture_label = deeplearning_predict(data, ture_label, model)
    for batch_idx in range(data.shape[0]):
        df_pred = pd.DataFrame(pred.cpu().detach().numpy()[batch_idx, :, :],
                                index=idx.detach().numpy()[batch_idx, :, 0],
                                columns=args.label_columns[:3])
        pred_list.append(df_pred)
df_predict = pd.concat(pred_list)
df_predict.sort_index(ascending=True, inplace=True, axis=0, key=None)
# Deduplication based on index, or taking the optimal value
# 根据index进行去重，或者说取最优数值
df_predict = df_predict.iloc[:, :3]
df_predict = df_predict.groupby(df_predict.index).median()
df_predict.rename(columns={'RHOB_pred': 'RHOB_reg', 'NPHI_pred': 'NPHI_reg', 'log_RD_pred': 'log_RD_reg'},
                    inplace=True)
df_all = test_df
df_dtw = df_all.join(df_predict, how='left', sort=False)
 # DTW
df_pred_y = DTW(df_dtw, args=args, mode=mode)
df_predict = pd.DataFrame(df_pred_y.values, columns=args.label_columns)
if mode != 'test':
    # dataframe->array->tensor
    tensor_label = torch.tensor(df_all[args.label_columns].values, dtype=torch.float32).unsqueeze(0)
    # tensor(200, 6) -> tensor(1, 200, 6)
    tensor_predict = torch.tensor(df_predict.values, dtype=torch.float32).unsqueeze(0)
    nmseloss = metrics_nmse(tensor_predict[:, :, :3], tensor_label[:, :, :3])
    madloss = metrics_mad(tensor_predict[:, :, 3:], tensor_label[:, :, 3:])
    pprint(args.log_file, f"{mode}-results after DTW: NMSELoss: {nmseloss:.5f}, MADLoss: {madloss:.5f}") 
df_all = df_all.reset_index().drop(columns=['index'], axis=1)  # Reset the index first, then delete the generated original backup index column - 先重置索引，后删除生成的原始备份index列
df_all[args.label_columns] = df_predict[args.label_columns]
df_all['log_RD'] = 10 ** df_all['log_RD']
df_all['log_RD_pred'] = 10 ** df_all['log_RD_pred']
df_all.rename(columns={'log_RD': 'RD', 'log_RD_pred': 'RD_pred', 'log_RD_dept_pred': 'RD_dept_pred'},
                inplace=True)
# save predict results
output_path = os.path.join(r'results/predict/', args.model, mode)
dir_exist(output_path)
wellnum_list = df_all['wellnum'].unique()
for wellnum in wellnum_list:
    df = df_all[df_all['wellnum'] == wellnum].copy()
    df = df.drop('wellnum', axis=1)
    df.to_csv(os.path.join(output_path, f'aligned_well_{wellnum}.csv'),
                index=False, encoding='utf_8_sig')

  3%|▎         | 4/143 [00:00<00:07, 19.04it/s]

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\2600609444.py:47: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='pchip', axis=0)  # Cubic interpolation (segmented cubic Hermite interpolation method) - used when the dataset presents a cumulative distribution - 立方插值(分段立方Hermite插值方法)-数据集呈现出累计分布时使用
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13452\260

### 5.6  Save the results to a zip package - 压缩包

In [94]:
# Save the results to a zip package
# 将结果保存成压缩包形式
shutil.make_archive(os.path.join(r'results/predict/', args.model, 'dreamstar_submission_1'),
                    'zip', output_path)

'f:\\python\\git\\gitee\\dreamstar\\results\\predict\\mWDN_gru\\dreamstar_submission_1.zip'